In [1]:
import numpy as np
import scipy.stats as stats
import random

import math
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import ScalarFormatter

In [7]:
D = 0.002 # thermal diffusion coefficient, cm^2/s
X = Y = 9 # size of vessel
T = 15 # total temperature to ramp over, C
delta_x = 1 # cm
delta_y = 1 # cm
delta_t = 100 # s

In [8]:
def temp_evolution():
    """temperature of idx, jdy, ndt"""
#     initialization
    temp_grid = np.zeros((X, Y))
    return temp_grid

In [10]:
temp_evolution()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])